<a href="https://colab.research.google.com/github/ThuyHaLE/ThuyHaLE/blob/main/Recommendation_production_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MY IDEA!**

I have a process called Plastic injection molding. A manufacturing process for producing parts by injecting molten material into a mold. The input material will be plastic (granules or powders). The output will be plastic parts. And we will use these parts to assemble them into a product. 

Here, I only care about the molding process. 

I have 49 machines (with different tonnage and models) and 237 molds (each mold will suit a particular machine and can produce a total of 783 various plastic parts). I will receive the production orders (POs) with the desired delivery date. 

I created a database that includes:
- Plastic information based on each plastic part/item (mold specification (name, code), capacity (psc/mold), optimized cycle time/shot (s), suitable machine tonnage list (the lowest tonnage will be a priority)).
- Mold information based on each plastic part/item (plastic types and required quantity (kg) for producing each 10000 pcs).
- Production report (total order list and order status).

Then I will use the database and the orders as the input to recommend the next most suitable production plan. And my output will be as follow:
- Production plan for each machine that includes an item list will be produced with an estimated lead time.
- Mold plan for each machine that includes a mold list will be produced with an estimated lead time.
- Plastic plan that includes a plastic type and quantity will be used for each machine/day.

##IMPORT SOME LIBRARY

In [33]:
import numpy as np
import pandas as pd
import datetime

from datetime import timedelta
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## INPUT:
* Production orders
### DATABASE:
* Plastic information based on each plastic part/item
* Mold information based on each plastic part/item
* Production report

In [34]:
#Import data (production info)
path ='/content/drive/MyDrive/Data science/Mine/Recommendation-Production Planning/DATABASE/PRO STATUS.xlsx'
data = pd.read_excel(path)
#Make sure that all values in PO NO. must be NOT null
pro_data = data[data['PO NO.'].notnull()]

#Import data (mold info)
path1 = '/content/drive/MyDrive/Data science/Mine/Recommendation-Production Planning/DATABASE/MOLD INFO.xlsx'
mold_data = pd.read_excel(path1)

#Import data (plastic info)
path2 = '/content/drive/MyDrive/Data science/Mine/Recommendation-Production Planning/DATABASE/PLASTIC INFO.xlsx'
plastic_data = pd.read_excel(path2)

#Import data (machine info)
path3 = '/content/drive/MyDrive/Data science/Mine/Recommendation-Production Planning/DATABASE/MACHINE INFO.xlsx'
machine_data = pd.read_excel(path3)

In [35]:
#Take a look at data
pro_data.head()

,PO RECEIVED DATE,PO NO.,PO DATE,ITEM CODE,ITEM NAME,OUTPUT DATE,MOLD STATUS,PO QUANTITY,PO REMAIN,MOLD NO.,MOLD NAME,MACHINE NO.,STARTED DATE,FINISHED DATE (ACTUAL),PRO. STATUS
0,2020-07-13,IM2007150,2020-07-13,10236M,AB-TP BODY,2020-08-15,M01,100000,0,20400IBE,AB-TP-BODY-M01,NO.47,2020-08-14,2020-08-26 00:00:00,FINISHED
1,2020-08-03,IM2008094,2020-08-03,10236M,AB-TP BODY,2020-09-01,M02,240000,0,20400IBE0200,AB-TP-BODY-M02,NO.47,2020-08-26,2020-09-28 00:00:00,FINISHED
2,2020-08-10,IM2008188,2020-08-10,10236M,AB-TP BODY,2020-09-01,M02,300000,0,20400IBE0200,AB-TP-BODY-M02,NO.47,2020-09-28,2020-10-06 00:00:00,FINISHED
3,2020-09-24,IM2009286,2020-09-25,10236M,AB-TP BODY,2020-11-01,M02,300000,0,20400IBE0200,AB-TP-BODY-M02,NO.49,2020-10-06,2021-04-09 00:00:00,FINISHED
4,2021-03-31,IM2103466,2021-03-24,10236M,AB-TP BODY,2021-04-15,M02,50000,0,20400IBE0200,AB-TP-BODY-M02,NO.49,2021-04-10,2021-04-14 00:00:00,FINISHED


In [36]:
mold_data.head()

,ITEM CODE,ITEM NAME,MOLD STATUS,MOLD NO.,MOLD NAME,CAVITY (STANDARD) PCS/MOLD,CAVITY (ACTUAL) PCS/MOLD,CYCLE (SUGGEST) S,TONNAGE TON
0,10236M,AB-TP BODY,M01,20400IBE,AB-TP-BODY-M01,4,3,36.0,130/180
1,10236M,AB-TP BODY,M02,20400IBE0200,AB-TP-BODY-M02,8,8,36.0,130/180
2,10238M,AB-TP LARGE CAP-020-IY,M01,20101IBE,AB-TP-LARGE-CAP-M01,4,4,21.0,50/100/130
3,10239M,AB-TP LARGE CAP-025-YW,M01,20101IBE,AB-TP-LARGE-CAP-M01,4,4,21.0,50/100/130
4,10240M,AB-TP LARGE CAP-026-YW,M01,20101IBE,AB-TP-LARGE-CAP-M01,4,4,21.0,50/100/130


In [37]:
plastic_data.head()

,ITEM CODE,ITEM NAME,MAIN CODE,MAIN NAME,MAIN QUAN. (KG/10000PCS),COLOR CODE,COLOR NAME,COLOR QUAN. (KG/10000PCS),ADDITION CODE,ADDITION,ADDITON QUAN. (KG/10000PCS)
0,10236M,AB-TP BODY,10050,PP-VN-J106MG-T.NL,50.358678,9917349000,MB-(3-28-33059)-ABTP-BODY-LG,0.923072,NaN,NaN,NaN
1,10238M,AB-TP LARGE CAP-020-IY,10049,PP-VN-J2023GR-T.NL,29.018533,9917349020,MB-(3-12-9635)-ABT-020-IY,0.175162,NaN,NaN,NaN
2,10239M,AB-TP LARGE CAP-025-YW,10049,PP-VN-J2023GR-T.NL,28.989339,9917349025,MB-(3-14-12381)-ABT-025-YW,0.204356,NaN,NaN,NaN
3,10240M,AB-TP LARGE CAP-026-YW,10049,PP-VN-J2023GR-T.NL,29.094436,9917349026,MB-(3-14-12382)-ABT-026-YW,0.099259,NaN,NaN,NaN
4,10241M,AB-TP LARGE CAP-027-BG,10049,PP-VN-J2023GR-T.NL,29.018533,9917349027,MB-(3-23-8309)-ABT-027-BG,0.175162,NaN,NaN,NaN


In [38]:
machine_data.head()

,MACHINE NO.,MANUFACTURER,MODEL,TONNAGE TON
0,NO.01,Niigata,MD50S,50
1,NO.02,Niigata,MD50S,50
2,NO.03,Toshiba,EC50ST,50
3,NO.04,Toshiba,EC50ST,50
4,NO.05,Toshiba,EC50ST,50


In [39]:
#Estimate full capacity (efficiency: 100%, loss: 0%) and actual capacity (efficiency: 85%, loss: 3%)
efficiency = 0.85
loss = 0.03
full_capacity = 3600/mold_data['CYCLE (SUGGEST) S']*mold_data['CAVITY (STANDARD) PCS/MOLD']
actual_capacity = (full_capacity*efficiency) - (full_capacity*loss) 

mold_data['FULL CAPACITY (ESTIMATED) PCS/HOUR'] = full_capacity
mold_data['ACTUAL CAPACITY (ESTIMATED) PCS/HOUR'] = actual_capacity

mold_data.head()

,ITEM CODE,ITEM NAME,MOLD STATUS,MOLD NO.,MOLD NAME,CAVITY (STANDARD) PCS/MOLD,CAVITY (ACTUAL) PCS/MOLD,CYCLE (SUGGEST) S,TONNAGE TON,FULL CAPACITY (ESTIMATED) PCS/HOUR,ACTUAL CAPACITY (ESTIMATED) PCS/HOUR
0,10236M,AB-TP BODY,M01,20400IBE,AB-TP-BODY-M01,4,3,36.0,130/180,400.000000,328.000000
1,10236M,AB-TP BODY,M02,20400IBE0200,AB-TP-BODY-M02,8,8,36.0,130/180,800.000000,656.000000
2,10238M,AB-TP LARGE CAP-020-IY,M01,20101IBE,AB-TP-LARGE-CAP-M01,4,4,21.0,50/100/130,685.714286,562.285714
3,10239M,AB-TP LARGE CAP-025-YW,M01,20101IBE,AB-TP-LARGE-CAP-M01,4,4,21.0,50/100/130,685.714286,562.285714
4,10240M,AB-TP LARGE CAP-026-YW,M01,20101IBE,AB-TP-LARGE-CAP-M01,4,4,21.0,50/100/130,685.714286,562.285714


## OUTPUT:
* Production plan for each machine
* Mold plan for each machine
* Plastic plan for each machine/day.

In [40]:
#Create a machine list
machine_no = []
for i in range(1,50):
  if i < 10:
    machine_no.append('NO.0'+ str(i))
  else:
    machine_no.append('NO.'+ str(i))

#Create output frame
output_plan = pd.DataFrame(columns = ['MACHINE NO.'])
output_plan['MACHINE NO.'] = machine_no

# PREPARE DATA
Divide "Production report" into 3 parts: 
* History (produced orders)
* Producing (producing orders)
* Pending (pending orders)

##Step 1: 
###Get data for "producing":
* Add some features such as full capacity, actual capacity, machine information
* Calculate lead time, remaining time, production progress, and finished date

###Use data frame named "producing" to:
* Update item names are produced on 49 machines into pro_plan
* Update item molds are produced on 49 machines into mold_plan
* Add plastic info based on item code into plastic_plan. Then calculate production quantity and plastic quantity for using in 24h for each machine

In [41]:
#Create a function to update production information about items and molds that are in progress, the daily amount of plastic to use for each machine
#With input = pro_data
def update_status_producing(data, mold_data, machine_data,output_plan, plastic_data):
  #There are 49 machines in the factory. So we must be sure that there will be 49 rows in the output data frame.
  producing_data = data[(data['PO REMAIN']>0) & (data['PRO. STATUS'] == 'PRODUCING')] #filter 
  #ascending sort by 'PRO. STATUS': Machine no. in use for each item
  producing_data = producing_data[['PO NO.', 'ITEM CODE', 'ITEM NAME','OUTPUT DATE', 'MOLD STATUS', 'PO QUANTITY', 'PO REMAIN', 'MACHINE NO.', 'STARTED DATE']].sort_values(by='MACHINE NO.') 
  #Add full capacity and actual capacity information based on item code and suggested mold status. 
  producing_data = producing_data.merge(mold_data[['ITEM CODE','MOLD STATUS', 'MOLD NO.', 'MOLD NAME','FULL CAPACITY (ESTIMATED) PCS/HOUR',
                                       'ACTUAL CAPACITY (ESTIMATED) PCS/HOUR']], how = 'left', on=['ITEM CODE','MOLD STATUS'])
  producing_data = producing_data.astype({'FULL CAPACITY (ESTIMATED) PCS/HOUR':'int','ACTUAL CAPACITY (ESTIMATED) PCS/HOUR':'int'})
  #Add machine information based on machine no.
  producing_data = producing_data.merge(machine_data[['MACHINE NO.','MODEL','TONNAGE TON']], how = 'left', on=['MACHINE NO.'])

  #Calculate lead time, remain time, production progress and finished date
  import datetime
  from datetime import timedelta
  lead_time = []
  for lead_t in (producing_data['PO QUANTITY']/(producing_data['ACTUAL CAPACITY (ESTIMATED) PCS/HOUR'])).to_list():
    lead_time.append(datetime.timedelta(hours=lead_t)) #calculate lead time
  remain_time = []
  for remain_t in (producing_data['PO REMAIN']/(producing_data['ACTUAL CAPACITY (ESTIMATED) PCS/HOUR'])).to_list():
    remain_time.append(datetime.timedelta(hours=remain_t)) #calculate remain time
  producing_data['LEAD TIME (DAYS)'] = lead_time
  producing_data['FINISHED DATE (ESTIMATED)'] = producing_data['STARTED DATE']+producing_data['LEAD TIME (DAYS)'] #calculate finished date
  producing_data['FINISHED DATE (ESTIMATED)'] = producing_data['FINISHED DATE (ESTIMATED)'].dt.strftime('%Y-%m-%d %H:%M:%S') #change form of finished date
  producing_data['PRO PROGRESS (%)'] = round(((producing_data['PO QUANTITY'] - producing_data['PO REMAIN'])*100/producing_data['PO QUANTITY']),2) #calculate production progress 
  producing_data['REMAIN TIME (DAYS)'] = remain_time

  producing_data1 = producing_data.copy()
  producing_data1['ITEM NAME & PO NO.'] = list(producing_data1['ITEM NAME']+' ('+producing_data1['PO NO.']+')')

  #Add item names that are produced by each machine into pro_plan
  pro_plan_update_producing = output_plan.merge(machine_data[['MODEL','TONNAGE TON','MACHINE NO.']], how = 'left', on=['MACHINE NO.'])
  pro_plan_update_producing['PRODUCING'] = producing_data1['ITEM NAME & PO NO.'].to_list()
  pro_plan_update_producing['PRODUCING LEAD TIME (DAYS)'] = producing_data1['REMAIN TIME (DAYS)'].to_list()

  #Add mold names are producing on each machine into mold_plan
  mold_plan_update_producing = output_plan.merge(machine_data[['MODEL','TONNAGE TON','MACHINE NO.']], how = 'left', on=['MACHINE NO.'])
  mold_plan_update_producing['PRODUCING'] = producing_data1['MOLD NAME'].to_list()
  mold_plan_update_producing['PRODUCING LEAD TIME (DAYS)'] = producing_data1['REMAIN TIME (DAYS)'].to_list()
  
  #Add plastic info base on item code
  plastic = producing_data1[['MACHINE NO.','PO NO.',	'ITEM CODE','ITEM NAME & PO NO.',	'ITEM NAME',	'PO REMAIN','FULL CAPACITY (ESTIMATED) PCS/HOUR']]
  plastic_merge = plastic.merge(plastic_data[['ITEM CODE','MAIN NAME','MAIN QUAN. (KG/10000PCS)', 'COLOR NAME','COLOR QUAN. (KG/10000PCS)', 
                                              'ADDITION','ADDITON QUAN. (KG/10000PCS)']], how = 'left', on=['ITEM CODE'])
  PRO_QUAN = [] #Calculate production quantity and plastic quantity for using in 24h for each machine
  for index, row in plastic_merge.iterrows():
    if row['PO REMAIN'] > (row['FULL CAPACITY (ESTIMATED) PCS/HOUR']*24):
      PRO_QUAN.append(row['FULL CAPACITY (ESTIMATED) PCS/HOUR']*24)
    else:
      PRO_QUAN.append(row['PO REMAIN'])
  plastic_merge['PRO QUAN. (PCS/DAY)'] = PRO_QUAN #production quantity
  plastic_merge['MAIN QUAN. (KG)'] = plastic_merge['MAIN QUAN. (KG/10000PCS)']/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #plastic quantity
  plastic_merge['COLOR QUAN. (G)'] = plastic_merge['COLOR QUAN. (KG/10000PCS)']*1000/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #color quantity (if any)
  plastic_merge['ADDITON QUAN. (G)'] = plastic_merge['ADDITON QUAN. (KG/10000PCS)']*1000/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #addition quantity (if any)
  #drop some feature for free space
  plastic_merge = plastic_merge.drop(['ADDITON QUAN. (KG/10000PCS)','MAIN QUAN. (KG/10000PCS)','COLOR QUAN. (KG/10000PCS)'],axis = 1) 
  #Add features from plastic into plastic plan
  plastic_plan_update_producing = output_plan.merge(machine_data[['MODEL','TONNAGE TON','MACHINE NO.']], how = 'left', on=['MACHINE NO.'])
  plastic_plan_update_producing['ITEM NAME & PO NO.'] = producing_data1['ITEM NAME & PO NO.'].to_list() #Item name
  plastic_plan_update_producing['PRO QUAN. (PCS/DAY)'] = plastic_merge['PRO QUAN. (PCS/DAY)'].to_list() #Production quantity
  plastic_plan_update_producing['PLASTIC'] = plastic_merge['MAIN NAME'].to_list() #Main plastic name
  plastic_plan_update_producing['P QUANTITY (KG/DAY)'] = plastic_merge['MAIN QUAN. (KG)'].to_list() #Main plastic quantity
  plastic_plan_update_producing['COLOR'] = plastic_merge['COLOR NAME'].to_list() #Color name
  plastic_plan_update_producing['C QUANTITY (G/DAY)'] = plastic_merge['COLOR QUAN. (G)'].to_list() #Color quantity
  plastic_plan_update_producing['ADDITION'] = plastic_merge['ADDITION'].to_list() #Addition name
  plastic_plan_update_producing['A QUANTITY (G/DAY)'] = plastic_merge['ADDITON QUAN. (G)'].to_list() #Addition quantity
  #Fill Nan in ['PLASTIC','COLOR','ADDITION'] by 'NONE'
  plastic_plan_update_producing[['PLASTIC','COLOR','ADDITION']] = plastic_plan_update_producing[['PLASTIC','COLOR','ADDITION']].fillna('NONE') 
  #Fill Nan in ['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)'] by 0
  plastic_plan_update_producing[['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)']] = plastic_plan_update_producing[['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)']].fillna(0)
  #Change type for ['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)'] to 'int'
  plastic_plan_update_producing = plastic_plan_update_producing.astype({'P QUANTITY (KG/DAY)':'int','C QUANTITY (G/DAY)':'int','A QUANTITY (G/DAY)':'int'})

  return producing_data, pro_plan_update_producing, mold_plan_update_producing, plastic_plan_update_producing

##Step 2: 
We will use "history", "producing", and "pending" to recommend suitable machines for molds used and never used before, then calculate the lead time for each machine
1. Based on mold infomation, since many molds are suitable for producing one item, we will use the best capacity in them as plan A.
2. We will recommend mold status for each item and add mold specification (mold code, mold name, cavity, cycle, capacity) in "pending".
3. We also will estimate lead time for each item

In [42]:
#Create a function to suggest the best suitaible machine for each item, based on history of production
#With input = pro_data

def best_suitable_machine_based_on_history(pro_data,machine_data,mold_data):
  #Get data for history
  hist = pro_data.loc[pro_data['PO REMAIN']==0].merge(machine_data[['MACHINE NO.','MODEL','TONNAGE TON']], how = 'left', on=['MACHINE NO.'])
  #Get data for pending
  pending = pro_data[(pro_data['PO REMAIN']>0) & (pro_data['PRO. STATUS'] != 'PRODUCING')][['PO NO.',	'ITEM CODE',	'ITEM NAME',	'OUTPUT DATE',	'PO QUANTITY']]
  
  #In some cases, there are many molds can produce one item. So we will choose the best capacity in them as plan A.
  #Filter out the best capacity as plan A
  mold_data_A = mold_data.drop(['TONNAGE TON'],axis=1).groupby('ITEM NAME').max().reset_index()
  #Suggest and add mold info of plan A based on item code
  pending_suggest_moldA = pending.merge(mold_data_A[['ITEM CODE', 'MOLD STATUS', 'MOLD NO.', 'MOLD NAME','CAVITY (STANDARD) PCS/MOLD',
                                     'CYCLE (SUGGEST) S', 'CAVITY (ACTUAL) PCS/MOLD','FULL CAPACITY (ESTIMATED) PCS/HOUR',
                                     'ACTUAL CAPACITY (ESTIMATED) PCS/HOUR']], how = 'left', on=['ITEM CODE'])
  lead_time_ = [] #Calculate lead time for each item
  for lead_ti in (pending_suggest_moldA['PO QUANTITY']/pending_suggest_moldA['ACTUAL CAPACITY (ESTIMATED) PCS/HOUR']).to_list():
    lead_time_.append(datetime.timedelta(hours=lead_ti)) #calculate lead time
  pending_suggest_moldA_leadtime = pending_suggest_moldA.copy()
  pending_suggest_moldA_leadtime['LEAD TIME (DAYS)'] = lead_time_
  pending_suggest_moldA_leadtime = pending_suggest_moldA_leadtime.astype({'FULL CAPACITY (ESTIMATED) PCS/HOUR':'int','ACTUAL CAPACITY (ESTIMATED) PCS/HOUR':'int'})
  pending_suggest_moldA_leadtime.head()

  #Create pending_matrix with index = mold name, columns = suitable machine
  pending_matrix = pd.DataFrame(index = pending_suggest_moldA_leadtime['MOLD NAME'].unique(), columns = ['SUITABLE MACHINES']).fillna('')
  #Based on history of production, create a machine dictionay with keys are mold names and values are suitaible machines
  machine_dictionary = {}
  hist_machine = hist.groupby('MOLD NAME')['MACHINE NO.'].unique() #index = MOLD NAME, column = list of machines used for these mold names before
  for mold_name_pending in pending_matrix.index: #loop for all mold name (matrix)
    if mold_name_pending in hist_machine.index: #if mold name (matrix) are in list of mold name (his_machine)
      machine_dictionary[mold_name_pending] = list(hist_machine[mold_name_pending]) #add {mold name (matrix): his_machine[mold name (matrix)]} into machine_dictionary
    else:
      machine_dictionary[mold_name_pending] = [] #else: add {mold name (matrix): []}
  #Based on machine_dictionary,
  for key in machine_dictionary: #loop for all mold names (machine_dictionary)
    for item in pending_matrix.index: #loop for all mold names (pending_matrix) 
      if key == item: #if mold name (machine_dictionary) == mold name (pending_matrix) 
          #fill column 'suitable machine' with corresponding machine_dictionary[key]
          pending_matrix.loc[key,'SUITABLE MACHINES'] = machine_dictionary[key] 
  #Make a copy of pending_matrix as suitable_matrix
  #For each item and each machine, if an item can produced by a machine, value [item, machine] will be 1 in suitable_matrix, else it will be 0
  #We will sum by each row to calculate number of machines will be suitable for each item
  suitable_matrix = pending_matrix.copy()
  for items, rows in suitable_matrix.iterrows(): #loop for each index and row in matrix
      for sui_machine in rows['SUITABLE MACHINES']: #rows['SUITABLE MACHINES'] is list => loop for each value in list
          suitable_matrix.at[items, sui_machine] = 1  #fill 1 into location [items, sui_machine] => else : NaN
  suitable_matrix = suitable_matrix.fillna(0) #fill NaN = 0
  suitable_matrix = suitable_matrix.drop('SUITABLE MACHINES',axis = 1) #drop column 'SUITABLE MACHINES'
  suitable_matrix['NUMBER_OF_SUITABLE_MACHINES'] = suitable_matrix.sum(axis=1) #total suitable machines will be sum of each row
  suitable_matrix = suitable_matrix.reindex(sorted(suitable_matrix.columns),axis=1) #sort column name by ascending
  suitable_matrix = suitable_matrix.sort_values('NUMBER_OF_SUITABLE_MACHINES') #sort total suitable machines by ascending

  #Make a copy of pending_matrix as machine_matrix
  machine_matrix = pending_matrix.copy().reset_index().rename(columns={'index':'MOLD NAME'})
  for machine in machine_no: #filter a data frame with each mold name and the frequency of each machine used before
    anpha = hist[hist['MACHINE NO.']==machine].groupby('MOLD NAME')['MACHINE NO.'].count().reset_index().rename(columns = {'MACHINE NO.':machine})
    machine_matrix = machine_matrix.merge(anpha, how = 'left', on=['MOLD NAME']) #merge machine_matrix with data frame above
  machine_matrix = machine_matrix.fillna(0).set_index('MOLD NAME').drop('SUITABLE MACHINES', axis =1)
  machine_matrix['MAX'] = machine_matrix.max(axis = 1) #find the maximum frequency of each mold name
  machine_matrix = machine_matrix[machine_matrix['MAX']!=0] #if maximum value = 0, it means these molds never used before. 
  #We will separate these molds for another step (suggest a suitable machine for new mold).

  #One mold can be used on many machines at many times. 
  #So we will suggest a suitable machine for each mold by choosing one machine that has the maximum frequency
  sui_machine = machine_matrix.idxmax(axis=1).reset_index() #return the index for the maximum value in each row, use axis=1 or ‘columns’.
  sui_machine.columns = ['MOLD NAME','MACHINE NO.']
  return hist, pending, pending_suggest_moldA_leadtime, sui_machine

In [43]:
#Create a function to update mold plan with pending data
def update_mold_plan_with_sui_machine(mold_plan_with_producing, best_sui_machine, pending_data_with_moldA):
  #Create a data frame with machine no. and total mold names will be used on it
  mold_list = best_sui_machine.groupby('MACHINE NO.')['MOLD NAME'].unique().reset_index()
  #Add sui_machine into pending
  pending_suggest_moldA_best_sui_machine = pending_data_with_moldA.merge(best_sui_machine, how = 'left', on=['MOLD NAME']).fillna('')
  #Get sum of lead time for each machine
  pending_leadtime = pending_suggest_moldA_best_sui_machine.groupby('MACHINE NO.')['LEAD TIME (DAYS)'].sum().reset_index()
  #Merge mold_plan with mold_list
  mold_plan_update_pending = mold_plan_with_producing.merge(mold_list, how = 'left', on=['MACHINE NO.'])
  #Add lead time for each machine into mold plan
  mold_plan_update_producing_pending = mold_plan_update_pending.merge(pending_leadtime, how = 'left', on=['MACHINE NO.'])
  mold_plan_update_producing_pending['MOLD NAME'] = mold_plan_update_producing_pending['MOLD NAME'].fillna('')
  mold_plan_update_producing_pending = mold_plan_update_producing_pending.rename(columns = {'LEAD TIME (DAYS)':'PENDING LEAD TIME (DAYS)'})
  mold_plan_update_producing_pending['PENDING LEAD TIME (DAYS)'] = mold_plan_update_producing_pending['PENDING LEAD TIME (DAYS)'].fillna(pd.Timedelta(seconds=0))
  mold_plan_update_producing_pending['TOTAL LEAD TIME (DAYS)'] = mold_plan_update_producing_pending['PENDING LEAD TIME (DAYS)'] + mold_plan_update_producing_pending['PRODUCING LEAD TIME (DAYS)']
  
  return mold_plan_update_producing_pending, pending_suggest_moldA_best_sui_machine

##Step 3:
We use "history", "producing", and "pending" to recommend suitable machines for molds never used before, then calculate the lead time for each machine

In [44]:
#Create a function recommend suitable machines based on machine tonnage
def suitable_machine_based_on_tonnage(pending_data, mold_data, mold_plan):
  #Get information of molds never used before
  mold = pending_data[pending_data['MACHINE NO.'] == '']
  #Merge these molds with mold_data to get suitable tonnage
  mold  = mold.merge(mold_data[['ITEM CODE','MOLD NAME','TONNAGE TON']], how = 'left', on=['MOLD NAME','ITEM CODE'])
  mold['TONNAGE TON'] = mold['TONNAGE TON'].astype(str)
  #Create a data frame with these mold and suitable tonnage
  mold_groupby_tonnage = mold.groupby('MOLD NAME')['TONNAGE TON'].unique().reset_index()
  #One mold can be used with many machine tonnages. But we will prioritize the minimum tonnage in them for safe mold.
  priorities = []
  for num in range(len(mold_groupby_tonnage['TONNAGE TON'])):
    priorities.append(mold_groupby_tonnage['TONNAGE TON'][num][0].split('/')[0])
  mold_pri_tonage = mold_groupby_tonnage
  mold_pri_tonage['PRIORITIES'] = priorities
  mold_pri_tonage['PRIORITIES'] = mold_pri_tonage['PRIORITIES'].astype(str)
  #We can see that some same tonnage machines have a lead time longer than others. 
  #So with the same tonnage, we will suggest the machine that has a shorter lead time in them.
  short_LT_tonnage = mold_plan.groupby('TONNAGE TON')['TOTAL LEAD TIME (DAYS)'].min().reset_index()
  best_sui_machine_short_LT = short_LT_tonnage.merge(mold_plan[['MACHINE NO.','TONNAGE TON','TOTAL LEAD TIME (DAYS)']],how='left',on=['TONNAGE TON','TOTAL LEAD TIME (DAYS)'])
  best_sui_machine_short_LT.columns = ['PRIORITIES','LEAD TIME (DAYS)','MACHINE NO.']
  best_sui_machine_short_LT['PRIORITIES'] = best_sui_machine_short_LT['PRIORITIES'].astype(str)
  best_sui_machine_tonnage = mold_pri_tonage.merge(best_sui_machine_short_LT[['PRIORITIES','MACHINE NO.']],how='left',on=['PRIORITIES'])
  best_sui_machine_tonnage = best_sui_machine_tonnage.drop(['TONNAGE TON',	'PRIORITIES'], axis = 1)
  return best_sui_machine_tonnage

#OPTIMALITY
The same machine models will have the same specifications. But after step 3, we can see that some machines with a lead time longer than the same specification others. Here we will adjust the mold list of each machine in order to the same specification machines will have a similar lead time for these machines.

##Step 4: 
Check over lead time for each machine.
- Over average lead time/machine
- Under average lead time/machine

In [45]:
#Create a function to check over lead time for each machine
def check_over_lead_time(mold_plan):
  #Check over capacity for each model
  check_over_model = mold_plan.groupby('MODEL')['TOTAL LEAD TIME (DAYS)'].sum().reset_index()
  check_over_model['NUM OF MACHINES'] = mold_plan.groupby('MODEL')['MACHINE NO.'].count().reset_index()['MACHINE NO.']
  check_over_model['LEAD TIME/MACHINE (DAYS)'] = check_over_model['TOTAL LEAD TIME (DAYS)'] / check_over_model['NUM OF MACHINES']
  check_over_model = check_over_model.sort_values('LEAD TIME/MACHINE (DAYS)',ascending = False)

  #Check over capacity for each tonnage
  check_over_tonnage = mold_plan.groupby('TONNAGE TON')['TOTAL LEAD TIME (DAYS)'].sum().reset_index()
  check_over_tonnage['NUM OF MACHINES'] = mold_plan.groupby('TONNAGE TON')['MACHINE NO.'].count().reset_index()['MACHINE NO.']
  check_over_tonnage['LEAD TIME/MACHINE (DAYS)'] = check_over_tonnage['TOTAL LEAD TIME (DAYS)'] / check_over_tonnage['NUM OF MACHINES']
  check_over_tonnage = check_over_tonnage.sort_values('LEAD TIME/MACHINE (DAYS)',ascending = False)

  #Use mold plan in step 4 and capacity status in step 5 to check over lead time for each machine
  check_over_LT = mold_plan.merge(check_over_model[['MODEL','LEAD TIME/MACHINE (DAYS)']],how='left',on=['MODEL'])
  check_over_LT = check_over_LT.set_index('MACHINE NO.') #use 'MACHINE NO.' as index
  check_over_LT.index.name = None
  #Count number of suitable molds for each machine
  num_of_mold = []
  for molds in check_over_LT['MOLD NAME']:
    num_of_mold.append(len(molds))
  check_over_LT['NUM OF MOLD'] = num_of_mold

  #In case lead time of each machine > average lead time => over lead time
  #If number of molds of a machine is 1 => We can consider using 2 or more molds for the shorter lead time.
  over_LT = check_over_LT[check_over_LT['TOTAL LEAD TIME (DAYS)']>=check_over_LT['LEAD TIME/MACHINE (DAYS)']].copy()
  #Check if we should use more molds for this item.
  nums = []
  for num in over_LT['NUM OF MOLD']:
    if num == 1:
      nums.append('YES')
    else: 
      nums.append('')
  over_LT['NEED MORE MOLD FOR THIS ITEM'] = nums

  #In case lead time of each machine < average lead time => over lead time
  under_LT = check_over_LT[check_over_LT['TOTAL LEAD TIME (DAYS)']<check_over_LT['LEAD TIME/MACHINE (DAYS)']]
  under_LT_ = under_LT[['LEAD TIME/MACHINE (DAYS)','TOTAL LEAD TIME (DAYS)']].copy()
  under_LT_['LT DIFF'] = under_LT_['LEAD TIME/MACHINE (DAYS)'] - under_LT_['TOTAL LEAD TIME (DAYS)']

  return check_over_model, check_over_tonnage, check_over_LT, over_LT, under_LT_

##Step 5:
In case "Over average lead time/machine", we will move some molds for these machines to other more suitable machines.

In [46]:
#Create a function to get priority molds for machines that are over-capacity, then make a list of molds that are moved out 
def modify_plan_for_OC_machines(pending_data,check_over_leadtime,mold_plan,under_leadtime_machines):
  #Get the lead time information for each mold name
  LT_bymold = pending_data.groupby('MOLD NAME')['LEAD TIME (DAYS)'].sum().reset_index()
  #Get information about machines and their mold list from over_LT
  over_LT_matrix = check_over_leadtime[['MOLD NAME']][check_over_leadtime['NUM OF MOLD']>=1].copy()
  #Create a over lead time matrix with columns = mold names, row = machines and values = the lead time of each mold
  for ix, ro in over_LT_matrix.iterrows():
    for value in ro['MOLD NAME']:
      over_LT_matrix.loc[ix,value] = LT_bymold['LEAD TIME (DAYS)'][LT_bymold['MOLD NAME']==value].to_list()
  over_LT_matrix = over_LT_matrix.drop('MOLD NAME',axis =1).fillna(pd.Timedelta(days=0))
  #Get molds that have the slowest lead time, we will use them as priority molds
  priority_molds = over_LT_matrix.idxmax(axis=1).reset_index()
  priority_molds.columns = ['MACHINE NO.', 'NEXT MOLD NAME']
  #Use priority molds and suitable molds for each machine to show the molds will remain after picking out priority molds for each machine
  mold_plan_a = priority_molds.merge(check_over_leadtime.rename_axis('MACHINE NO.').reset_index()[['MACHINE NO.','MOLD NAME']],
                      how = 'left', on = 'MACHINE NO.')
  mold_plan_a = mold_plan_a.set_index('NEXT MOLD NAME')
  mold_plan_a.index.name = None
  #show the molds will remain after picking out priority molds for each machine
  diff = []
  for idx, rec in mold_plan_a.iterrows():
    diff.append(list(set(rec['MOLD NAME'].tolist()) - set([idx])))
  mold_plan_a['DIFF'] = diff
  mold_plan_a = mold_plan_a.reset_index()
  mold_plan_a.columns = ['MOLD NAME',	'MACHINE NO.',	'MOLD NAME LIST', 'MOLD DIFF']
  #add lead time for priority molds and lead time for each machine
  mold_plan_b = mold_plan_a.merge(LT_bymold, how = 'left', on = 'MOLD NAME')
  mold_plan_b = mold_plan_b.merge(check_over_leadtime.rename_axis('MACHINE NO.').reset_index()[['MACHINE NO.', 'LEAD TIME/MACHINE (DAYS)']],
                                            how = 'left', on = 'MACHINE NO.')
  mold_plan_b = mold_plan_b.set_index('MACHINE NO.')
  mold_plan_b.index.name = None
  #In case lead time for priority molds < for each machine => add more molds into mold plan 
  mold_plan_c = mold_plan_b[mold_plan_b['LEAD TIME (DAYS)']<mold_plan_b['LEAD TIME/MACHINE (DAYS)']].drop('MOLD NAME LIST', axis= 1)
  mold_plan_c['LT DIFF'] = (mold_plan_c['LEAD TIME/MACHINE (DAYS)']) - (mold_plan_c['LEAD TIME (DAYS)'])
  mold_plan_d = mold_plan_c[['MOLD DIFF']].copy()
  #Create a over lead time matrix with columns = mold names, row = machines and values = the lead time of each mold
  for ixs, ros in mold_plan_d.iterrows():
    for values in ros['MOLD DIFF']:
      mold_plan_d.loc[ixs,values] = LT_bymold['LEAD TIME (DAYS)'][LT_bymold['MOLD NAME']==values].to_list()
  mold_plan_d = mold_plan_d.drop('MOLD DIFF',axis =1)
  mold_plan_d = mold_plan_d.fillna(pd.Timedelta(days=0))
  mold_plan_d = mold_plan_d.merge(mold_plan_c[['LT DIFF']],left_index=True, right_index=True)
  #Based on over lead time matrix to suggest the next priority (if any)
  next_priority = {}
  for mno in mold_plan_d.index:
    machi_mol = {}
    for cl in mold_plan_d.drop('LT DIFF',axis=1).columns:
      if mold_plan_d.loc[mno, cl] > datetime.timedelta(hours=0):
        if mold_plan_d.loc[mno, 'LT DIFF'] > mold_plan_d.loc[mno, cl]:
          machi_mol[cl] = mold_plan_d.loc[mno, 'LT DIFF'] - mold_plan_d.loc[mno, cl]
    next_priority[mno] = machi_mol
  next_priority_ = pd.DataFrame.from_dict(next_priority).dropna(how='all', axis=1)
  next_after_priority = next_priority_.idxmin().reset_index()
  next_after_priority.columns = ['MACHINE NO.', 'NEXT PRIORITY']
  #Get priority molds and next priority molds
  mold_plan_e = mold_plan_a[['MACHINE NO.','MOLD NAME']].rename(columns = {'MOLD NAME':'PRIORITY'}).merge(next_after_priority,how= 'left', on='MACHINE NO.')
  mold_plan_e = mold_plan_e.fillna('NONE')
  mold_plan_e
  #Add priority molds and next priority molds into mold plan
  modified_mold_plan = mold_plan.merge(mold_plan_e,how = 'left', on = 'MACHINE NO.')
  modified_mold_plan[['PRIORITY', 'NEXT PRIORITY']] = modified_mold_plan[['PRIORITY', 'NEXT PRIORITY']].fillna('')

  #Add next priority into mold_plan
  mold_plan_f = mold_plan_a[['MACHINE NO.','MOLD DIFF']].merge(next_after_priority, how='left',on='MACHINE NO.').fillna('').set_index('NEXT PRIORITY')
  mold_plan_f.index.name = None
  #show the molds will remain after picking out priority molds for each machine
  diffe = []
  for idxs, recos in mold_plan_f.iterrows():
    diffe.append(list(set(recos['MOLD DIFF']) - set([idxs])))
  mold_plan_f['DIFF'] = diffe
  mold_plan_f = mold_plan_f.reset_index()
  mold_plan_f.columns = ['MOLD NAME',	'MACHINE NO.',	'MOLD NAME LIST', 'MOLD DIFF 2']
  #Get the break list to recommend other suitable machines
  break_list_ = mold_plan_f[['MACHINE NO.','MOLD DIFF 2']].merge(
      machine_data[['MACHINE NO.','MODEL']],how='left',on='MACHINE NO.').groupby('MODEL')['MOLD DIFF 2'].sum().reset_index()
  fil = []
  for dif in break_list_['MOLD DIFF 2'].to_list():
    if len(dif) > 0: fil.append(dif)
    else: fil.append('')
  break_list_['MOLD DIFF 2'] = fil
  #suitable machines will be in machine list that has leadtime is under average leadtime
  suitable_machine_list = under_leadtime_machines.rename_axis('MACHINE NO.').reset_index().drop(
      ['LEAD TIME/MACHINE (DAYS)',	'TOTAL LEAD TIME (DAYS)',	'LT DIFF'], axis = 1).merge(
      machine_data[['MACHINE NO.', 'MODEL',	'TONNAGE TON']], how = 'left', on = 'MACHINE NO.').groupby(
          'MODEL')['MACHINE NO.'].unique().reset_index()
  break_list = break_list_[break_list_['MOLD DIFF 2']!=''].set_index('MODEL')
  break_list.index.name = None
  #recommend other suitable machine for the break list
  break_ls = {}
  for models, mold_list in break_list.iterrows():
    for mold_names in mold_list['MOLD DIFF 2']:
      break_ls[mold_names] = models,LT_bymold['LEAD TIME (DAYS)'][LT_bymold['MOLD NAME']==mold_names].to_list()
  break_ls = pd.DataFrame.from_dict(break_ls,orient='index').reset_index()
  break_ls.columns = ['MOLD NAME','MODEL','LEAD TIME']
  break_ls = break_ls.merge(suitable_machine_list,how='left',on='MODEL')

  return modified_mold_plan, break_ls

In [47]:
#Create a function to update new pending data with new suitable machine information
def update_mold_plan_with_new_priority(pending_data, new_priority, mold_plan, production_plan):
  new_pending = pending_data.merge(new_priority, how='left', on = 'MOLD NAME').fillna('')
  changed = []
  for id, change in new_pending.iterrows():
    if change['NEW MACHINE NO.'] == '':
      changed.append(new_pending['MACHINE NO.'][id])
    else:
      changed.append(new_pending['NEW MACHINE NO.'][id])
  new_pending['CHANGED MACHINE NO.'] = changed
  new_pending_b = new_pending.drop(['MACHINE NO.',	'NEW MACHINE NO.'],axis=1).rename(columns = {'CHANGED MACHINE NO.':'MACHINE NO.'})
  new_pending_c = new_pending_b.replace('BREAK', '')
  #update new mold plan by new pending data
  mold_plan_a = mold_plan.copy().merge(
      new_pending_c.groupby('MOLD NAME')['LEAD TIME (DAYS)'].sum().reset_index().merge(
          new_pending_c[['MOLD NAME','MACHINE NO.']].drop_duplicates(), how = 'left', on = 'MOLD NAME').sort_values(
              by=['LEAD TIME (DAYS)','MACHINE NO.'],ascending = False).groupby('MACHINE NO.')['MOLD NAME'].unique().reset_index(),
              how = 'left', on = 'MACHINE NO.')
  mold_plan_b = mold_plan_a.merge(new_pending_c.groupby('MACHINE NO.')['LEAD TIME (DAYS)'].sum().reset_index(), how = 'left', on = 'MACHINE NO.')
  mold_plan_b = mold_plan_b.rename(columns = {'LEAD TIME (DAYS)':'PENDING LEAD TIME (DAYS)'})
  mold_plan_b['TOTAL LEAD TIME (DAYS)'] = mold_plan_b['PRODUCING LEAD TIME (DAYS)'] + mold_plan_b['PENDING LEAD TIME (DAYS)']

  #update new production plan by new pending data
  new_pending_d = new_pending_c.copy()
  new_pending_d['ITEM NAME & PO NO.'] = list(new_pending_d['ITEM NAME']+' ('+new_pending_d['PO NO.']+')')
  production_plan_a = production_plan.copy().merge(
      new_pending_d.groupby('ITEM NAME')['LEAD TIME (DAYS)'].sum().reset_index().merge(
          new_pending_d[['ITEM NAME','MACHINE NO.']].drop_duplicates(), how = 'left', on = 'ITEM NAME').sort_values(
              by=['LEAD TIME (DAYS)','MACHINE NO.'],ascending = False).groupby('MACHINE NO.')['ITEM NAME'].unique().reset_index(),
              how = 'left', on = 'MACHINE NO.')
  production_plan_b = production_plan_a.merge(
      new_pending_d.groupby('ITEM NAME & PO NO.')['LEAD TIME (DAYS)'].sum().reset_index().merge(
          new_pending_d[['ITEM NAME & PO NO.','MACHINE NO.']].drop_duplicates(), how = 'left', on = 'ITEM NAME & PO NO.').groupby(
              'MACHINE NO.')['ITEM NAME & PO NO.'].unique().reset_index(), how = 'left', on = 'MACHINE NO.')
  production_plan_b = production_plan_b.merge(new_pending_d.groupby('MACHINE NO.')['LEAD TIME (DAYS)'].sum().reset_index(), how = 'left', on = 'MACHINE NO.')
  production_plan_b = production_plan_b.rename(columns = {'LEAD TIME (DAYS)':'PENDING LEAD TIME (DAYS)'})
  production_plan_b['TOTAL LEAD TIME (DAYS)'] = production_plan_b['PRODUCING LEAD TIME (DAYS)'] + production_plan_b['PENDING LEAD TIME (DAYS)']
  production_plan_b = production_plan_b.drop('ITEM NAME', axis = 1)
  return new_pending_d, mold_plan_b, production_plan_b

In [48]:
#Create a function that recommend the best suitable machine for the break list
def best_suitable_machine_for_break_list(break_list,under_leadtime_machine):
  #Recommend best suitable machine for molds in break list
  pri_machines = break_list.copy().set_index('MOLD NAME').drop('MODEL',axis = 1)
  #the suitable machine will in machine list that has remain lead time < average lead time
  for mold_name, machine in pri_machines.drop('LEAD TIME',axis = 1).iterrows():
    for machine_numa in machine['MACHINE NO.']:
      if machine_numa in under_leadtime_machine['LT DIFF']:
        pri_machines.loc[mold_name,machine_numa] = under_leadtime_machine['LT DIFF'][machine_numa]
  pri_machines1 = pri_machines.fillna(pd.Timedelta(seconds=0)).drop(['MACHINE NO.'],axis=1)
  leadtimes = []
  for leadtime in pri_machines['LEAD TIME']:
    leadtimes.append(leadtime[0])
  pri_machines1['LEAD TIME'] = leadtimes
  pri_machines1['MAX'] = pri_machines1.drop(['LEAD TIME'],axis=1).sum(axis=1)
  pri_machines1['DIFF'] = pri_machines1['MAX'] - pri_machines1['LEAD TIME']
  #the best suitable machine will in machine that has longest remain lead time
  priority = pri_machines1[pri_machines1['DIFF']>=datetime.timedelta(hours = 0)].drop(
      ['LEAD TIME','MAX','DIFF'],axis=1).idxmax(axis=1).reset_index().append(
          pri_machines1[pri_machines1['DIFF']<datetime.timedelta(hours = 0)].reset_index()[['MOLD NAME']]).fillna('BREAK')
  priority.columns = ['MOLD NAME', 'NEW MACHINE NO.']
  return priority

In [49]:
#Create a function to recommend mold plan in case there is priority list
def recommend_mold_plan_with_priority_list(new_priority_list, pending_data, mold_plan):
  default_priority = []
  for mn in mold_plan['MOLD NAME']:
    if len(mn) >= 1:
      default_priority.append(mn[0])
    else:
      default_priority.append('NONE')

  if new_priority_list == []:
    new_priority = pd.DataFrame(default_priority, columns=['MOLD NAME']).merge(pending_data[['MOLD NAME','MACHINE NO.']].drop_duplicates(),
                                                                                how = 'left', on = 'MOLD NAME').groupby(
                                                                                    'MACHINE NO.')['MOLD NAME'].unique().reset_index().rename(
                                                                                        columns = {'MOLD NAME': 'NEW PRIORITY MOLD NAME'})

  else:
    new_priority = pd.DataFrame(new_priority_list, columns=['MOLD NAME']).merge(pending_data[['MOLD NAME','MACHINE NO.']].drop_duplicates(),
                                                                                how = 'left', on = 'MOLD NAME').groupby('MACHINE NO.')['MOLD NAME'].unique().reset_index().rename(
                                                                                    columns = {'MOLD NAME': 'NEW PRIORITY MOLD NAME'})

  priority_mold = pd.DataFrame(default_priority, columns=['MOLD NAME']).merge(pending_data[['MOLD NAME','MACHINE NO.']].drop_duplicates(),
                        how = 'left', on = 'MOLD NAME').groupby('MACHINE NO.')['MOLD NAME'].unique().reset_index().rename(
                            columns = {'MOLD NAME': 'DEFAULT PRIORITY MOLD NAME'}).merge(new_priority, how = 'left', on = 'MACHINE NO.')
  next_molds = []
  for ind, row in priority_mold.iterrows():
    if len(row['NEW PRIORITY MOLD NAME']) == 0:
      next_molds.append([row['DEFAULT PRIORITY MOLD NAME'][0]])
    else:
      next_molds.append([row['NEW PRIORITY MOLD NAME'][0]])

  mold_plan_1 = mold_plan.copy()
  mold_plan_1['NEXT MOLD'] = next_molds
  diff = []
  for idxs, recs in mold_plan_1.iterrows():
    if len(recs['MOLD NAME']) >= len(recs['NEXT MOLD']):
      if len(recs['NEXT MOLD']) >= 1:
        if list(set(recs['MOLD NAME']) - set(recs['NEXT MOLD'])) == []:
          diff.append('NONE')
        else: diff.append(list(set(recs['MOLD NAME']) - set(recs['NEXT MOLD'])))

  mold_plan_1['PENDING LIST'] = diff
  new_mold_plan = mold_plan_1.reset_index()[['MACHINE NO.',	'MODEL',	'TONNAGE TON',	'PRODUCING',	'PRODUCING LEAD TIME (DAYS)',	
                                    'NEXT MOLD',	'PENDING LIST', 'PENDING LEAD TIME (DAYS)',	'TOTAL LEAD TIME (DAYS)']]
  return new_mold_plan

In [50]:
#Create a function to recommend production plan in case there is priority list
def recommend_production_plan_with_priority_list(new_priority_list, pending_data, production_plan):
  default_priority = []
  for mn in production_plan['ITEM NAME & PO NO.']:
    if len(mn) >= 1:
      default_priority.append(mn[0])
    else:
      default_priority.append('NONE')
  
  if new_priority_list == []:
    new_priority = pd.DataFrame(default_priority, columns=['ITEM NAME & PO NO.']).merge(pending_data[['ITEM NAME & PO NO.','MACHINE NO.']],
                        how = 'left', on = 'ITEM NAME & PO NO.').groupby('MACHINE NO.')['ITEM NAME & PO NO.'].unique().reset_index().rename(
                            columns = {'ITEM NAME & PO NO.': 'NEW PRIORITY ITEM NAME & PO NO.'})

  else: new_priority = pd.DataFrame(new_priority_list, columns=['ITEM NAME & PO NO.']).merge(pending_data[['ITEM NAME & PO NO.','MACHINE NO.']],
                        how = 'left', on = 'ITEM NAME & PO NO.').groupby('MACHINE NO.')['ITEM NAME & PO NO.'].unique().reset_index().rename(
                            columns = {'ITEM NAME & PO NO.': 'NEW PRIORITY ITEM NAME & PO NO.'})
  default_priority = []
  for mn in production_plan['ITEM NAME & PO NO.']:
    if len(mn) >= 1:
      default_priority.append(mn[0])
    else:
      default_priority.append('NONE')
  priority_item = pd.DataFrame(default_priority, columns=['ITEM NAME & PO NO.']).merge(pending_data[['ITEM NAME & PO NO.','MACHINE NO.']].drop_duplicates(),
                        how = 'left', on = 'ITEM NAME & PO NO.').groupby('MACHINE NO.')['ITEM NAME & PO NO.'].unique().reset_index().rename(
                            columns = {'ITEM NAME & PO NO.': 'DEFAULT PRIORITY ITEM NAME & PO NO.'}).merge(new_priority, how = 'left', on = 'MACHINE NO.')
  next_items = []
  for ind, row in priority_item.iterrows():
    if len(row['NEW PRIORITY ITEM NAME & PO NO.']) == 0:
      next_items.append([row['DEFAULT PRIORITY ITEM NAME & PO NO.'][0]])
    else:
      next_items.append([row['NEW PRIORITY ITEM NAME & PO NO.'][0]])

  production_plan_1 = production_plan.copy()
  production_plan_1['NEXT ITEM NAME & PO NO.'] = next_items
  diff = []
  for idxs, recs in production_plan_1.iterrows():
    if len(recs['ITEM NAME & PO NO.']) >= len(recs['NEXT ITEM NAME & PO NO.']):
      if len(recs['NEXT ITEM NAME & PO NO.']) >= 1:
        if list(set(recs['ITEM NAME & PO NO.']) - set(recs['NEXT ITEM NAME & PO NO.'])) == []:
          diff.append('NONE')
        else: diff.append(list(set(recs['ITEM NAME & PO NO.']) - set(recs['NEXT ITEM NAME & PO NO.'])))

  production_plan_1['PENDING LIST'] = diff
  new_production_plan = production_plan_1.reset_index()[['MACHINE NO.',	'MODEL',	'TONNAGE TON',	'PRODUCING',	'PRODUCING LEAD TIME (DAYS)',	
                                    'NEXT ITEM NAME & PO NO.',	'PENDING LIST', 'PENDING LEAD TIME (DAYS)',	'TOTAL LEAD TIME (DAYS)']]

  return new_production_plan

In [51]:
#Create a function to recommend a item list that need to use 2 molds since over capacity
def recommed_item_need_2_molds(over_listime_list, pending_data):
  over_listime = over_listime_list['MOLD NAME'][over_listime_list['NEED MORE MOLD FOR THIS ITEM'] == 'YES'].rename_axis(
      'MACHINE NO.').reset_index()
  over_listime_items = []
  for mold_name in over_listime['MOLD NAME']:
    over_listime_items.append(mold_name[0])
  over_listime_item = over_listime.copy()
  over_listime_item['MOLD NAME'] = over_listime_items
  over_listime_item_list = over_listime_item.merge(pending_data[['MOLD NAME','MACHINE NO.','ITEM NAME']].drop_duplicates(),
                                        how = 'left', on = ['MACHINE NO.','MOLD NAME'])[['ITEM NAME','MOLD NAME']].merge(
                                            pending_data.groupby('ITEM NAME')['PO NO.'].unique(), how = 'left', on = 'ITEM NAME').merge(
                                                pending_data.groupby('ITEM NAME')['PO QUANTITY'].sum(), how = 'left', on = 'ITEM NAME').merge(
                                                    (mold_data.groupby('ITEM NAME')['MOLD NAME'].count().reset_index().rename(
                                                        columns = {'MOLD NAME': 'NUMBER OF MOLD'})),how = 'left', on = 'ITEM NAME').rename(
                                                            columns = {'MOLD NAME': 'MOLD NAME (PLAN)'})
                                                        
  over_listime_item_list_fn = over_listime_item_list[over_listime_item_list['NUMBER OF MOLD'] > 1].drop(
      'NUMBER OF MOLD', axis = 1).reset_index(drop=True).merge((mold_data.groupby('ITEM NAME')['MOLD STATUS'].unique().reset_index().rename(
                                                    columns = {'MOLD STATUS': 'USABLE MOLD LIST'})),how = 'left', on = 'ITEM NAME').merge(
                                                        (mold_data.groupby('ITEM NAME')['ACTUAL CAPACITY (ESTIMATED) PCS/HOUR'].max().
                                                         reset_index().rename(columns = {'ACTUAL CAPACITY (ESTIMATED) PCS/HOUR': 
                                                                                         'AVARAGE CAPACITY (PCS/HOUR)'})),
                                                        how = 'left', on = 'ITEM NAME') [['ITEM NAME',	'PO NO.',	'PO QUANTITY', 
                                                                                          'MOLD NAME (PLAN)','USABLE MOLD LIST',
                                                                                          'AVARAGE CAPACITY (PCS/HOUR)']]
  return over_listime_item_list_fn

In [52]:
#Create a function to calculate production quantity and plastic quantity for using in 24h for each machine
#In case the lead time of producing items less than 1 day
def prepare_plastic_for_next_items(production_plan,plastic_data):
  need_prepare_plastic = production_plan[production_plan['PRODUCING LEAD TIME (DAYS)']<datetime.timedelta(days=1)][['MACHINE NO.', 'NEXT ITEM NAME & PO NO.']].reset_index(drop=True)
  items = []
  POs = []
  for lis in need_prepare_plastic['NEXT ITEM NAME & PO NO.']:
    items.append(lis[0].replace(')','').split(' (')[0])
    POs.append(lis[0].replace(')','').split(' (')[1])
  need_prepare_plastic['PO NO.'] = POs
  need_prepare_plastic['ITEM NAME'] = items
  need_prepare_plastic1 = need_prepare_plastic.merge(
      final_pending[['PO NO.',	'ITEM CODE',	'ITEM NAME', 'MOLD NAME', 'PO QUANTITY']], how= 'left', on = ['PO NO.','ITEM NAME']).merge(
          mold_data[['ITEM NAME','MOLD NAME','FULL CAPACITY (ESTIMATED) PCS/HOUR']], how= 'left', on = ['MOLD NAME','ITEM NAME'])
  need_prepare_plastic2 = need_prepare_plastic1[['MACHINE NO.','NEXT ITEM NAME & PO NO.', 'PO NO.',	'ITEM CODE',	'ITEM NAME',	'PO QUANTITY','FULL CAPACITY (ESTIMATED) PCS/HOUR']]
  plastic_merge = need_prepare_plastic2.merge(plastic_data[['ITEM CODE','MAIN NAME','MAIN QUAN. (KG/10000PCS)', 'COLOR NAME','COLOR QUAN. (KG/10000PCS)', 
                                                'ADDITION','ADDITON QUAN. (KG/10000PCS)']], how = 'left', on=['ITEM CODE'])
  PRO_QUAN = [] #Calculate production quantity and plastic quantity for using in 24h for each machine
  for index, row in plastic_merge.iterrows():
    if row['PO QUANTITY'] > (row['FULL CAPACITY (ESTIMATED) PCS/HOUR']*24):
      PRO_QUAN.append(row['FULL CAPACITY (ESTIMATED) PCS/HOUR']*24)
    else:
      PRO_QUAN.append(row['PO QUANTITY'])

  plastic_merge['PRO QUAN. (PCS/DAY)'] = PRO_QUAN #production quantity
  plastic_merge['MAIN QUAN. (KG)'] = plastic_merge['MAIN QUAN. (KG/10000PCS)']/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #plastic quantity
  plastic_merge['COLOR QUAN. (G)'] = plastic_merge['COLOR QUAN. (KG/10000PCS)']*1000/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #color quantity (if anyplastic_merge['ADDITON QUAN. (G)'] = plastic_merge['ADDITON QUAN. (KG/10000PCS)']*1000/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #addition quantity (if any)
  plastic_merge['ADDITON QUAN. (G)'] = plastic_merge['ADDITON QUAN. (KG/10000PCS)']*1000/10000*plastic_merge['PRO QUAN. (PCS/DAY)'] #addition quantity (if any)
  #drop some feature for free space
  plastic_merge = plastic_merge.drop(['ADDITON QUAN. (KG/10000PCS)','MAIN QUAN. (KG/10000PCS)','COLOR QUAN. (KG/10000PCS)'],axis = 1) 
  #Add features from plastic into plastic plan
  pending_plastic_plan = need_prepare_plastic.drop(['PO NO.',	'ITEM NAME'], axis = 1)
  #pending_plastic_plan['NEXT ITEM NAME & PO NO.'] = need_prepare_plastic['NEXT ITEM NAME & PO NO.'].to_list() #Item name
  pending_plastic_plan['PRO QUAN. (PCS/DAY)'] = plastic_merge['PRO QUAN. (PCS/DAY)'].to_list() 
  pending_plastic_plan['PLASTIC'] = plastic_merge['MAIN NAME'].to_list() #Main plastic name
  pending_plastic_plan['P QUANTITY (KG/DAY)'] = plastic_merge['MAIN QUAN. (KG)'].to_list() #Main plastic quantity
  pending_plastic_plan['COLOR'] = plastic_merge['COLOR NAME'].to_list() #Color name
  pending_plastic_plan['C QUANTITY (G/DAY)'] = plastic_merge['COLOR QUAN. (G)'].to_list() #Color quantity
  pending_plastic_plan['ADDITION'] = plastic_merge['ADDITION'].to_list() #Addition name
  pending_plastic_plan['A QUANTITY (G/DAY)'] = plastic_merge['ADDITON QUAN. (G)'].to_list() #Addition quantity
  #Fill Nan in ['PLASTIC','COLOR','ADDITION'] by 'NONE'
  pending_plastic_plan[['PLASTIC','COLOR','ADDITION']] = pending_plastic_plan[['PLASTIC','COLOR','ADDITION']].fillna('NONE') 
  #Fill Nan in ['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)'] by '0'
  pending_plastic_plan[['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)']] = pending_plastic_plan[['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)']].fillna(0)
  #Change type for ['P QUANTITY (KG/DAY)','C QUANTITY (G/DAY)','A QUANTITY (G/DAY)'] to 'int'
  pending_plastic_plan = pending_plastic_plan.astype({'P QUANTITY (KG/DAY)':'int','C QUANTITY (G/DAY)':'int','A QUANTITY (G/DAY)':'int'})
  return pending_plastic_plan

#MODEL

In [53]:
#Step 1: Get information of producing items, update production information about items and molds that are in progress
data_1 = pro_data
producing_data, pro_plan_with_producing, mold_plan_with_producing, plastic_plan_with_producing = update_status_producing(data_1, mold_data, machine_data,output_plan, plastic_data)

In [54]:
#Step 2: 
#Get information of production history and pending items, recommend suitable machine for each item
history, pending_data, pending_data_with_moldA, sui_machine  = best_suitable_machine_based_on_history(data_1,machine_data,mold_data)
#Recommend mold plan for pending data (molds used before) - we will use the molds that haves largest capacity
mold_plan_with_bestsui_moldA, pending_data_with_bestsui_moldA = update_mold_plan_with_sui_machine(mold_plan_with_producing, sui_machine, pending_data_with_moldA)

In [55]:
#Step 3: 
#Recommend suitable machines for the items that never produced before based on machine tonnage
best_sui_machine_firstime = suitable_machine_based_on_tonnage(pending_data_with_bestsui_moldA, mold_data, mold_plan_with_bestsui_moldA)
#Create the best suitable machines list for all items in pending (inclue first time items)
all_mold_pending = best_sui_machine_firstime.append(sui_machine)
#Recommend mold plan for the pending data (inclue items used and never used before)
mold_plan_with_bestsui_all, pending_data_with_bestsui_all = update_mold_plan_with_sui_machine(mold_plan_with_producing, all_mold_pending, pending_data_with_moldA)

In [71]:
#Step 4:
#Check if the machines will be over-capacity based on machine mode, machine tonnage, average leadtime of each machine after we recommended the best suitable for each item 
check_over_model, check_over_tonnage, check_over_average_leadtime, over_average_leadtime, under_average_leadtime = check_over_lead_time(mold_plan_with_bestsui_all)

In [ ]:
#Step 5: 
#For the machines that be over-average-lead time, we will recommend the new mold plan with the best suitable items for these machines
#And get items list that be not suitable for these machines, so that we can recommend other suitable machines for these items
modified_mold_plan, break_list = modify_plan_for_OC_machines(pending_data_with_bestsui_all,check_over_average_leadtime,mold_plan_with_producing,under_average_leadtime)

In [73]:
#Recommend the best suitable machine for the break list
new_priority = best_suitable_machine_for_break_list(break_list,under_average_leadtime)
#Update new pending data with new suitable machine information
new_pending, new_mold_plan, new_pro_plan = update_mold_plan_with_new_priority(pending_data_with_bestsui_all, new_priority, mold_plan_with_producing, pro_plan_with_producing)
#Recommend the best suitable machine for the break list
new_best_sui_machine = suitable_machine_based_on_tonnage(new_pending, mold_data, new_mold_plan)
#Update new pending data with new suitable machine information
final_pending, final_mold_plan, final_pro_plan = update_mold_plan_with_new_priority(new_pending,
                                                                                    new_best_sui_machine.copy().rename(columns = {'MACHINE NO.':'NEW MACHINE NO.'}), 
                                                                                    mold_plan_with_producing, pro_plan_with_producing)

In [74]:
#Check if the machines will be over-capacity based on machine mode, machine tonnage, average leadtime of each machine after we recommended the best suitable for each item 
final_check_over_model, final_check_over_tonnage, final_check_over_leadtime, final_over_leadtime, final_under_leadtime = check_over_lead_time(final_mold_plan)

In [75]:
#Recommend mold plan in case there is priority list 
priority_mold_list = []
new_mold_plan_with_priority = recommend_mold_plan_with_priority_list(priority_mold_list, final_pending, final_mold_plan)
#Recommend production plan in case there is priority list
priority_item_list = []
new_production_plan_with_priority = recommend_production_plan_with_priority_list(priority_item_list, final_pending, final_pro_plan)
#Recommend a item list that need to use 2 molds since over capacity
need_2_molds_list = recommed_item_need_2_molds(final_over_leadtime, final_pending)


In [76]:
#Calculate production quantity and plastic quantity for using in 24h for each machine
#In case the lead time of producing items less than 1 day
pending_plastic_plan = prepare_plastic_for_next_items(new_production_plan_with_priority,plastic_data)

#OUTPUT

In [77]:
#Final_mold_plan
new_mold_plan_with_priority.head()

,MACHINE NO.,MODEL,TONNAGE TON,PRODUCING,PRODUCING LEAD TIME (DAYS),NEXT MOLD,PENDING LIST,PENDING LEAD TIME (DAYS),TOTAL LEAD TIME (DAYS)
0,NO.01,MD50S,50,PN-RR-GEAR-M03,1 days 20:09:18.068615,[CT-CA-SPRING-STOP-M02],[CT-CF-PRINTER-HEAD-5.0-M03],10 days 23:25:23.170731,12 days 19:34:41.239346
1,NO.02,MD50S,50,CT-CC-CORE-M02,1 days 20:54:31.854157,[CT-PAX-BASE-M01],NONE,26 days 18:47:04.390245,28 days 15:41:36.244402
2,NO.03,EC50ST,50,PN-R-REEL-M02-MEW,2 days 04:44:15.072464,[CT-CAX-CARTRIDGE-BASE-M01],NONE,19 days 20:21:10.739593,22 days 01:05:25.812057
3,NO.04,EC50ST,50,PN-RR-PRINTER-HEAD-M01-NEW,2 days 17:17:31.546392,[CT-CAX-BASE-COVER-M01],NONE,35 days 23:49:16.097564,38 days 17:06:47.643956
4,NO.05,EC50ST,50,PN-CAS-COVER-M01,0 days 16:56:56.949153,[PN-MAS-FLANGE-M01],"[PN-CAS-REEL-M01, PN-IR-REEL-M01, PN-EP-HEAD-C...",31 days 22:00:21.951217,32 days 14:57:18.900370


In [78]:
#Final_production_plan
new_production_plan_with_priority.head()

,MACHINE NO.,MODEL,TONNAGE TON,PRODUCING,PRODUCING LEAD TIME (DAYS),NEXT ITEM NAME & PO NO.,PENDING LIST,PENDING LEAD TIME (DAYS),TOTAL LEAD TIME (DAYS)
0,NO.01,MD50S,50,PN-CR GEAR (IM2105149),1 days 20:09:18.068615,[CT-CA SPRING-STOP VER.1 (IM2104098)],"[CT-CA SPRING-STOP VER.1 (IM2105084), CT-CF PR...",10 days 23:25:23.170731,12 days 19:34:41.239346
1,NO.02,MD50S,50,CT-CC CORE SILVER VER.1 (IM2104375),1 days 20:54:31.854157,[CT-PAX BASE DARK BLUE (IM2104236)],"[XCCT-PAX5CL BASE BLUE (IM2104326), CT-PAX BAS...",26 days 18:47:04.390245,28 days 15:41:36.244402
2,NO.03,EC50ST,50,PN-CR RELL V1 (IM2105235),2 days 04:44:15.072464,[CT-CAX CARTRIDGE-BASE (IM2104104)],"[CT-CAX CARTRIDGE-BASE (IM2104417), CT-CAX CAR...",19 days 20:21:10.739593,22 days 01:05:25.812057
3,NO.04,EC50ST,50,PN-CR PRINTER HEAD WHITE (IM2104165),2 days 17:17:31.546392,[CT-CAX BASE-COVER (IM2104107)],"[CT-CAX BASE-COVER (IM2104423), CT-CAX BASE-CO...",35 days 23:49:16.097564,38 days 17:06:47.643956
4,NO.05,EC50ST,50,PN-CAS COVER CLEAR BLUE B (IM2104292),0 days 16:56:56.949153,[PN-CAS REEL NATURAL WHITE (IM2104283)],"[PN-ER CAP SKY BLUE (IM2104477), PN-ER CAP GRE...",31 days 22:00:21.951217,32 days 14:57:18.900370


In [79]:
#Producing_plastic_plan
plastic_plan_with_producing.head()

,MACHINE NO.,MODEL,TONNAGE TON,ITEM NAME & PO NO.,PRO QUAN. (PCS/DAY),PLASTIC,P QUANTITY (KG/DAY),COLOR,C QUANTITY (G/DAY),ADDITION,A QUANTITY (G/DAY)
0,NO.01,MD50S,50,PN-CR GEAR (IM2105149),92160,POM NW-02-NL,63,NONE,0,NONE,0
1,NO.02,MD50S,50,CT-CC CORE SILVER VER.1 (IM2104375),131640,ABS PA-757,52,MFABS-2737-SILVER-,2191,NONE,0
2,NO.03,EC50ST,50,PN-CR RELL V1 (IM2105235),36360,POM-RE-F20-NL,68,NONE,0,NONE,0
3,NO.04,EC50ST,50,PN-CR PRINTER HEAD WHITE (IM2104165),17040,POM NW-02-NL,34,POM-WEF-19379-WHITE-,1422,NONE,0
4,NO.05,EC50ST,50,PN-CAS COVER CLEAR BLUE B (IM2104292),10000,PC-RE-OSTC-BLUE-T.NL,18,PLAST-TRANS-BLUE-MFPC-VN5957-R1-19,854,NONE,0


In [80]:
#Pending_plastic_plan
pending_plastic_plan.head()

,MACHINE NO.,NEXT ITEM NAME & PO NO.,PRO QUAN. (PCS/DAY),PLASTIC,P QUANTITY (KG/DAY),COLOR,C QUANTITY (G/DAY),ADDITION,A QUANTITY (G/DAY)
0,NO.05,[PN-CAS REEL NATURAL WHITE (IM2104283)],17280.000000,POM-RE-F20-NL,64,NONE,0,NONE,0
1,NO.08,[CT-CA CLUTCH VER.1 (IM2104413)],86400.000000,ABS PA-757,13,MASTERBATCH-MFABS-943-BLACK,410,NONE,0
2,NO.15,[CT-CC CORE L. BLUE VER.1 (IM2104376)],20000.000000,ABS PA-757,8,MASTERBATCH-MFABS-968-BLUE,166,NONE,0
3,NO.17,[CT-CA PRINTER-HEAD 4.2MM GREEN (IM2105090)],18189.473684,POM-M9044-NL,22,MASTERBATCH-GEF-49151-GREEN,924,NONE,0
4,NO.19,[CT-CAX REEL (IM2104113)],30052.173913,POM-RE-F20-NL,87,NONE,0,NONE,0


In [81]:
#Require_2_molds_list
need_2_molds_list.head()

,ITEM NAME,PO NO.,PO QUANTITY,MOLD NAME (PLAN),USABLE MOLD LIST,AVARAGE CAPACITY (PCS/HOUR)
0,CT-CAX REEL,"[IM2104113, IM2104114, IM2104425, IM2104426, I...",600000,CT-CAX-REEL-M02,"[M01, M02]",1026.782609
1,"PN-MS8.4 SMALL GEAR PN-MS,MK,MSF,MSM 8.4","[IM2104139, IM2104353, IM2104354, IM2104395, I...",775000,PN-MS-8.4SMALL-GEAR-M05,"[M04, M05]",787.200000
2,PN-MSM UPPER CASE PINK,"[IM2105022, IM2105023]",100000,PN-MS8.4-UPPER-CASE-M06,"[M01, M05, M06]",590.400000
3,UPPER CASE 8.4,"[IM2104350, IM2104390, IM2104391, IM2104392, I...",490000,PN-MS8.4-UPPER-CASE-M06,"[M01, M05, M06]",590.400000


#EXPORT THE RESULTS

In [82]:
#Use to_excel function and specify the sheet_name and index, to store the dataframe in specified sheet
with pd.ExcelWriter("/content/drive/MyDrive/Data science/Mine/Recommendation-Production Planning/RESUTLS/results.xlsx") as writer:
  producing_data.to_excel(writer, sheet_name="Producing_data", index=False)
  history.to_excel(writer, sheet_name="History_data", index=False)
  pending_data.to_excel(writer, sheet_name="Pending_data", index=False)
  final_pending.to_excel(writer, sheet_name="Final_pending", index=False)
  new_mold_plan_with_priority.to_excel(writer, sheet_name="Final_mold_plan", index=False)
  new_production_plan_with_priority.to_excel(writer, sheet_name="Final_production_plan", index=False)
  plastic_plan_with_producing.to_excel(writer, sheet_name="Producing_plastic_plan", index=False)
  pending_plastic_plan.to_excel(writer, sheet_name="Pending_plastic_plan", index=False)
  need_2_molds_list.to_excel(writer, sheet_name="Require_2_molds_list", index=False)

In [83]:
#from google.colab import files
#files.download('"/content/drive/MyDrive/Data science/Recommendation-Production Planning/RESUTLS/results.xlsx"')